In [ ]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import os
from PIL import Image
import pathlib
import csv
import keras
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianNoise
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import metrics
import warnings
import csv
import tensorflow as tf
import time
from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
import splitfolders


splitfolders.ratio('/content/drive/MyDrive/vowel_experimental_image', output="./vowel_data", seed=1337, ratio=(.8, .2)) # default values

train_data_generator = ImageDataGenerator(
        rescale=1./255, 
        shear_range=0.5, 
        zoom_range=0.5,
        width_shift_range=0.5, 
        height_shift_range=0.5,
        brightness_range=[0.1, 10],
        featurewise_center=True, 
        featurewise_std_normalization=True,
        samplewise_std_normalization=True,
        horizontal_flip=True) 

test_data_generator = ImageDataGenerator(
        rescale=1./255, 
        shear_range=0.5, 
        zoom_range=0.5,
        width_shift_range=0.5, 
        height_shift_range=0.5,
        brightness_range=[0.1, 10],
        featurewise_center=True, 
        featurewise_std_normalization=True,
        samplewise_std_normalization=True,
        horizontal_flip=True)


test = train_data_generator.flow_from_directory(
        '/content/drive/MyDrive/vowel_experimental_image',
        target_size=(64, 64),
        batch_size= 8,
        class_mode='categorical',
        shuffle = False)

training = test_data_generator.flow_from_directory(
        '/content/drive/MyDrive/vowel_experimental_image',
        target_size= (64, 64),
        batch_size= 8,
        class_mode='categorical',
        shuffle = False)

model = Sequential()
model.add(Conv2D(64, (3, 3), strides=(2, 2), input_shape=(64,64,3)))
model.add(MaxPooling2D((2, 2), strides=(2,2)))
model.add(BatchNormalization())
model.add(GaussianNoise(0.125))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(MaxPooling2D((2, 2), strides=(2,2)))
model.add(BatchNormalization())
model.add(GaussianNoise(0.125))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

epochs = 100
batch_size = 8
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])


model.fit_generator(
        training,
        steps_per_epoch= 1,
        epochs= 100,
        validation_data=training,
        validation_steps=1)
model.evaluate_generator(generator=test, steps=1)


Copying files: 196 files [00:00, 378.45 files/s]


Found 196 images belonging to 2 classes.
Found 196 images belonging to 2 classes.
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 31, 31, 64)        1792      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 15, 64)        256       
_________________________________________________________________
gaussian_noise (GaussianNois (None, 15, 15, 64)        0         
_________________________________________________________________
activation_13 (Activation)   (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 13, 13, 32)        18464     
______________________________________

[0.7115454077720642, 0.375]

In [ ]:
test.reset()
pred = model.predict_generator(test, steps=25, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

labels = (training.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:200]
filenames=test.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(labels)
print(results)

25/25 [==============================] - 4s 124ms/step
{0: 'negative', 1: 'positive'}
                      Filename Predictions
0    negative/vowel-a (10).png    positive
1    negative/vowel-a (11).png    negative
2    negative/vowel-a (12).png    negative
3    negative/vowel-a (13).png    positive
4    negative/vowel-a (14).png    negative
..                         ...         ...
191      positive/vowel-a4.png    positive
192      positive/vowel-a5.png    positive
193      positive/vowel-a6.png    positive
194      positive/vowel-a8.png    positive
195      positive/vowel-a9.png    positive

[196 rows x 2 columns]


In [ ]:
test.reset()
pred = model.predict_generator(test, steps=25, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

labels = (training.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:200]
filenames=test.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(labels)
print(results)

25/25 [==============================] - 3s 126ms/step
{0: 'negative', 1: 'positive'}
                      Filename Predictions
0    negative/vowel-a (10).png    positive
1    negative/vowel-a (11).png    positive
2    negative/vowel-a (12).png    negative
3    negative/vowel-a (13).png    negative
4    negative/vowel-a (14).png    negative
..                         ...         ...
191      positive/vowel-a4.png    positive
192      positive/vowel-a5.png    positive
193      positive/vowel-a6.png    positive
194      positive/vowel-a8.png    positive
195      positive/vowel-a9.png    positive

[196 rows x 2 columns]
